In [1]:
from data_load import mixture
import tensorflow as tf
import numpy as np
path = '/Users/smaity/projects/mnist-label-shift/MNIST'

In [2]:
x_source, y_source = mixture(12000, 0.9, flatten=False)
x_target, y_target = mixture(1000, 0.2, flatten=False)
x_source.shape, y_source.shape

((12000, 28, 28), (12000,))

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20)

Epoch 1/20
375/375 [==============================] - 1s 3ms/step - loss: 4.0238 - accuracy: 0.8888
Epoch 2/20
375/375 [==============================] - 1s 3ms/step - loss: 0.7613 - accuracy: 0.9281
Epoch 3/20
375/375 [==============================] - 1s 3ms/step - loss: 0.3656 - accuracy: 0.9455
Epoch 4/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2103 - accuracy: 0.9550
Epoch 5/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1665 - accuracy: 0.9606
Epoch 6/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1236 - accuracy: 0.9656
Epoch 7/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1211 - accuracy: 0.9659
Epoch 8/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0849 - accuracy: 0.9747
Epoch 9/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0729 - accuracy: 0.9783
Epoch 10/20
375/375 [==============================] - 1s 3ms/step - loss: 0.0702 - accuracy: 0.9775

In [4]:
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.9543 - accuracy: 0.8460


[0.9542704820632935, 0.8460000157356262]

In [5]:
y_predict = model.predict_classes(x_target)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [6]:
from sklearn import metrics
m = 12000
reg = 0
prop_source = 0.9
confusion_matrix = metrics.confusion_matrix(model.predict_classes(x_source),y_source, labels=[0,1])/m
prop_target = np.mean(model.predict_classes(x_target))
    
xi = np.array([1-prop_target,prop_target])
w = np.matmul(np.linalg.inv(confusion_matrix + reg*np.diag([1, 1])),xi)
prop_targets = w*np.array([1-prop_source, prop_source])
if prop_targets[0] <0:
    prop_targets[0] = 0
if prop_targets[1] <0:
    prop_targets[1] = 0
prop_targets = prop_targets/np.sum(prop_targets)

In [7]:
prop_targets

array([0.67731092, 0.32268908])

In [8]:
sample_weight_0 = prop_targets[0]/0.1
sample_weight_1 = prop_targets[1]/0.9
sample_weight = sample_weight_0 * (1-y_source) + sample_weight_1 * y_source


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(3, 2, input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(x_source, y_source, epochs=20, sample_weight = sample_weight)


Epoch 1/20
375/375 [==============================] - 0s 1ms/step - loss: 9.4257 - accuracy: 0.6656
Epoch 2/20
375/375 [==============================] - 0s 1ms/step - loss: 2.2766 - accuracy: 0.7797
Epoch 3/20
375/375 [==============================] - 0s 1ms/step - loss: 1.0326 - accuracy: 0.8269
Epoch 4/20
375/375 [==============================] - 0s 1ms/step - loss: 0.6629 - accuracy: 0.8606
Epoch 5/20
375/375 [==============================] - 0s 1ms/step - loss: 0.5728 - accuracy: 0.8674
Epoch 6/20
375/375 [==============================] - 0s 1ms/step - loss: 0.4340 - accuracy: 0.8827
Epoch 7/20
375/375 [==============================] - 0s 1ms/step - loss: 0.3701 - accuracy: 0.8884
Epoch 8/20
375/375 [==============================] - 0s 1ms/step - loss: 0.2384 - accuracy: 0.9116
Epoch 9/20
375/375 [==============================] - 0s 1ms/step - loss: 0.2152 - accuracy: 0.9204
Epoch 10/20
375/375 [==============================] - 0s 1ms/step - loss: 0.1732 - accuracy: 0.9281

In [9]:
model.evaluate(x_target, y_target, verbose=2)

32/32 - 0s - loss: 0.2875 - accuracy: 0.9330


[0.2875029742717743, 0.9330000281333923]